In [24]:
from PIL import Image
from random import*
# fonction qui transforme une image en une liste de pixels rgba
def convert_image_pixel(file):
    image = Image.open(file)
    image = image.convert('RGBA')
    liste_pixels = list(image.getdata())
    return liste_pixels

# fonction qui encode un message dans une image
def traitement_image(file, texte):
    im = Image.open(file)
    im = im.convert('RGBA')
    w_img = (im.size)[0]
    h_img = (im.size)[1]
    if len(texte)>((w_img-8)/3):
        return("Le message est trop long pour être encodé dans l'image, veuillez raccourcir votre message ou bien fournir une image ayant une résolution plus élevée")
    # la variable binaire est un string de binaire correspondant au texte donne en argument
    binaire=''.join(format(ord(i), '08b') for i in texte)
    liste_binaire=[int(i) for i in binaire]
        # liste_binaire est une liste contenant des entier. chaque entier est un bit en binaire
    nb_pixels = len(liste_binaire)//4*1.5

    coord_occ=[]
    for k in range(nb_pixels):
        # on place les bits en partant du pixel en haut a gauche. on incremente de un a un vers la droite jusqua toucher
        # la limite droite, on revient ensuite a gauche en descendant d'une ligne. donc l'abscisse est de k modulo la
        # largeur de l'image, et l'ordonnee est la division entiere de k par la largeur de l'image.
        coordinate = k % w_img, k//w_img
        tpl = im.getpixel(coordinate)
        couleur = list(tpl)
        couleur_bin = [int((bin(couleur[0]))[2:10]), int((bin(couleur[1]))[2:10]), int((bin(couleur[2]))[2:10]),int((bin(couleur[3]))[2:10])] #couleur_bin est une liste. les valeurs prises sont 'couleur' en binaire, string, auquel on enleve le '0b' du debut(avec un slice), et qu'on reconvertit en string
        for i in range(4):
            if liste_binaire[4*k+i] != couleur_bin[i] % 2:
                couleur_bin[i] = str((couleur_bin[i]//10)*10+liste_binaire[4*k+i])
                couleur_bin[i] = int(couleur_bin[i], 2)
            else:
                couleur_bin[i] = int(str(couleur_bin[i]), 2)
        couleur_bin = tuple(couleur_bin)
        im.putpixel(((k % w_img), (k//w_img)), couleur_bin)
    # la varibale liste_nb_pixels contient la valeur en binaire sur 32 bits du nombre de pixels a decoder
    liste_nb_pixels = list(bin(nb_pixels))
    liste_nb_pixels.pop(0)
    liste_nb_pixels.pop(0)
    liste_nb_pixels.reverse()
    for i in range(len(liste_nb_pixels)):
        liste_nb_pixels[i]=int(liste_nb_pixels[i])
    while len(liste_nb_pixels)<32:
        liste_nb_pixels.append(0)
    # je laisse la liste a l'envers car je vais prendre les pixels en partant du bit de poids faible
    for i in range(8):
        # j'utilise ces operations sur x et y pour que le nombre de pixel stocke soit ecrit meme si l'image fait moins de 8 pixels de large
        x = -1 - i % w_img
        y = -1-i//w_img
        couleur_pixel=list(im.getpixel((x, y)))
        for k in range(1, 5):
            if couleur_pixel[-k]%2==1 and liste_nb_pixels[0]%2==0:
                couleur_pixel[-k]-=1
            elif couleur_pixel[-k]%2==0 and liste_nb_pixels[0]%2==1:
                couleur_pixel[-k]+=1
            del(liste_nb_pixels[0])
        im.putpixel([x, y], tuple(couleur_pixel))
    im.show()
    n_file=str(file)
    name=''
    i=0
    while n_file[i]!='.':
        name+=n_file[i]
        i+=1
    name+=' (encodé).png'
    im.save(name)




# fonction qui decode le nombre de pixels utilises dans l'encodage


def nb_pixels_encodes(image):
    # recuperer la liste des pixels rgba decimal d'une image
    liste_pixels = convert_image_pixel(image)
    # initialiser une liste qui va contenir les bits necessaires pour avoir
    # le nombre de pixels utilises dans l'encodage (a l'envers)
    liste_bit = []
    for i in range(1, 9):
        for k in range(1, 5):
            bit = liste_pixels[-i][-k] % 2
            liste_bit.append(bit)
    # remettre la liste des bits a l'endroit
    liste_bit.reverse()
    # transformer la liste en une chaine de caractere
    nombre_bin = ''
    for bit in liste_bit:
        nombre_bin += str(bit)
    # transformer le nombre de pixel en decimal
    nombre_pixels_utilises = int(nombre_bin, 2)
    return nombre_pixels_utilises


# fonction qui decode le message cache dans l'image
def decodage(image):
    # liste des pixels (en rgba) qui constituent l'image
    liste_pixels_rgba = convert_image_pixel(image)
    # nombre de pixels utilises pour encoder le messsage
    nb_pixels = nb_pixels_encodes(image)
    # initialiser une liste qui contient les bits necessaires pour le decodage
    message_bin = []
    # recuperer les bits necessaires pour le decodage
    for i in range(nb_pixels):
        for k in range(4):
            bit = liste_pixels_rgba[i][k] % 2
            message_bin.append(bit)
    # initialiser une nouvelle liste qui va contenir les bits regroupes en 8
    message_bin_8 = []
    while len(message_bin) > 0:
        lettre_bin = ''
        for i in range(8):
            lettre_bin += str(message_bin[i])
        message_bin_8.append(lettre_bin)
        del message_bin[0:8]
    # initialiser une nouvelle liste qui va contenir les nombres decimaux correspondant a chaque caractere
    message_ascii = []
    for bits8 in message_bin_8:
        message_ascii.append(int(bits8, 2))
    # initialiser une nouvelle liste qui va contenir les caracteres du message encodes
    message_unicode = []
    for nbr in message_ascii:
        message_unicode.append(chr(nbr))
    # transformer la liste en une chaine de caractere
    message = ''
    for caractere in message_unicode:
        message += caractere
    return message


# programme principal
traitement_image('image groupe.jpg', 'On est les gagnants')
print(decodage('image groupe (encodé).png'))


In [49]:
def lmfao(image):
    return convert_image_pixel(image)
lmfao('/Users/roulaabifadel/Downloads/blanc10x10 (encodé).png')

[(254, 255, 254, 254),
 (254, 255, 254, 255),
 (255, 254, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (254, 255, 254, 255),
 (254, 254, 255, 255),
 (255, 254, 254, 254),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (254, 255, 254, 254),
 (254, 255, 255, 255),
 (254, 254, 254, 254),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255, 255, 255),
 (255, 255,

In [64]:
def lol(image):
    listeimage=convert_image_pixel(image)
    listefinale=[]
    for i in range(len(listeimage)):
        listx=[]
        for j in range(4):
            listx.append(bin(listeimage[i][j]))
        listefinale.append(listx)    
        
    return listefinale

lol('/Users/roulaabifadel/Downloads/blanc10x10 (encodé).png')

[['0b11111110', '0b11111111', '0b11111110', '0b11111110'],
 ['0b11111110', '0b11111110', '0b11111110', '0b11111111'],
 ['0b11111110', '0b11111110', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111110', '0b11111111', '0b11111110', '0b11111110'],
 ['0b11111110', '0b11111110', '0b11111111', '0b11111110'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111110'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'],
 ['0b11111111', '0b11111111', '0b11111111', '0b11111111'

In [18]:
from PIL import Image
from random import*

#fonction qui va verifier les coordonnees du pixel que l'on tente d'occuper. si celles-ci depassent le bord de l'image ou si cet emplacement est deja occupe, on en genere un nouveau
def verification_coords(coordx, coordy, w_img, h_img, coord_occ):
    while coordx>=w_img:
        coordx-=w_img
        coordy+=1
    if coordy>=h_img:
        coordy=0
    while (coordx,coordy) in coord_occ:
        coordx=coordx+1
        if coordx>=w_img:
            coordx=0
            coordy+=1
            if coordy>=h_img:
                coordy=0
    return coordx, coordy
# fonction qui encode un message dans une image
def traitement_image(file, texte):
    im = Image.open(file)
    im = im.convert('RGBA')
    w_img = (im.size)[0]
    h_img = (im.size)[1]
    if len(texte)>((w_img*h_img)//3):
        return("Le message est trop long pour être encodé dans l'image, veuillez raccourcir votre message ou bien fournir une image ayant une résolution plus élevée")
    # la variable binaire est un string de binaire correspondant au texte donne en argument
    binaire=''.join(format(ord(i), '08b') for i in texte)
    liste_binaire=[int(i) for i in binaire]
        # liste_binaire est une liste contenant des entier. chaque entier est un bit en binaire
    nb_lettres = len(liste_binaire)//8
    coord_occ=[]
    nb_pixels_morts=(w_img*h_img)%3
    if nb_pixels_morts>0:
        coord_occ.append((w_img-1,h_img-1))
        if nb_pixels_morts>1:
            coord_occ.append((w_img-2,h_img-1))
    coordx= 0
    coordy= 0
    for k in range(nb_lettres):
        # on place les bits en partant du pixel en haut a gauche. on incremente de un a un vers la droite jusqua toucher
        # la limite droite, on revient ensuite a gauche en descendant d'une ligne. donc l'abscisse est de k modulo la
        # largeur de l'image, et l'ordonnee est la division entiere de k par la largeur de l'image.
        for z in range(3):
            if z!=0:
                coordx+=1
            if coordx>=w_img:
                coordx=0
                coordy+=1
                if coordy>=h_img:
                    coordy=0
            tpl = im.getpixel((coordx,coordy))
            couleur = list(tpl)
            couleur_bin = [int((bin(couleur[0]))[2:10]), int((bin(couleur[1]))[2:10]), int((bin(couleur[2]))[2:10]),int((bin(couleur[3]))[2:10])] #couleur_bin est une liste. les valeurs prises sont 'couleur' en binaire, string, auquel on enleve le '0b' du debut(avec un slice), et qu'on reconvertit en string            
            if z==2:
                saut=randint(1,15)
                sautbin='{:04b}'.format(saut)
                listesautbin=[int(m) for m in sautbin]
                if k==nb_lettres-1:
                    saut=0
                    listesautbin=[0 for _ in range(4)]
                for i in range(4):
                    if listesautbin[i] != couleur_bin[i] % 2:
                        couleur_bin[i] = str((couleur_bin[i]//10)*10+listesautbin[i])
                        couleur_bin[i] = int(couleur_bin[i], 2)
                    else:
                        couleur_bin[i]=int(str(couleur_bin[i]), 2)
            else:
                for i in range(4):
                    if liste_binaire[4*(2*k+z)+i] != couleur_bin[i] % 2:
                        couleur_bin[i] = str((couleur_bin[i]//10)*10+liste_binaire[4*(2*k+z)+i])
                        couleur_bin[i] = int(couleur_bin[i], 2)
                    else:
                        couleur_bin[i] = int(str(couleur_bin[i]), 2)
            coord_occ.append((coordx,coordy))
            couleur_bin = tuple(couleur_bin)
            im.putpixel((coordx, coordy), couleur_bin)
            if z==2 and saut!=0:
                coordx+=(saut-1)*3+1
                old_coordy=coordy
                coordx,coordy= verification_coords(coordx, coordy, w_img, h_img, coord_occ)
                if coordy<old_coordy:
                    coordx,coordy=0,0
                    coordx,coordy=verification_coords(coordx, coordy, w_img, h_img, coord_occ)
    im.show()
    n_file=str(file)
    name=''
    i=0
    while n_file[i]!='.':
        name+=n_file[i]
        i+=1
    name+=' (encodé).png'
    im.save(name)
    print(liste_binaire)
    print(coord_occ)
    return coord_occ

#fontion qui transforme un string contenant un nombre binaire en un entier decimal
def bin_a_dec(binaire):
    Lbinaire=list(binaire)
    Lbinaire.reverse()
    decimal=0
    for i in range(len(Lbinaire)):
        decimal+=int(Lbinaire[i])*2**i
    return decimal



# fonction qui decode le message cache dans l'image
def decodage(image):
    im = Image.open(image)
    im = im.convert('RGBA')
    w_img = (im.size)[0]
    h_img = (im.size)[1]
    saut=15
    coordx=0
    coordy=0
    coord_occ=[]
    nb_pixels_morts=(w_img*h_img)%3
    if nb_pixels_morts>0:
        coord_occ.append((w_img-1,h_img-1))
        if nb_pixels_morts>1:
            coord_occ.append((w_img-2,h_img-1))
    message_bin=[]
    while saut!=0:
        for z in range(3):
            if z!=0:
                coordx+=1
            if coordx>=w_img:
                coordx=0
                coordy+=1
                if coordy>=h_img:
                    coordy=0
            tpl = im.getpixel((coordx,coordy))
            couleur = list(tpl)
            bits = [str(couleur[0]%2), str(couleur[1]%2), str(couleur[2]%2), str(couleur[3]%2)]
            if z==2:
                bits_str=''.join(bits)
                saut=bin_a_dec(bits_str)
            else:
                for m in bits:
                    message_bin.append(m)
            coord_occ.append((coordx,coordy))
            if z==2 and saut!=0:
                coordx+=(saut-1)*3+1
                old_coordy=coordy
                coordx,coordy= verification_coords(coordx, coordy, w_img, h_img, coord_occ)
                if coordy<old_coordy:
                    coordx,coordy=0,0
                    coordx,coordy=verification_coords(coordx, coordy, w_img, h_img, coord_occ)
    message_bin_8 = []
    while len(message_bin) > 0:
        lettre_bin = ''
        for i in range(8):
            lettre_bin += str(message_bin[i])
        message_bin_8.append(lettre_bin)
        del message_bin[0:8]
    # initialiser une nouvelle liste qui va contenir les nombres decimaux correspondant a chaque caractere
    message_ascii = []
    for bits8 in message_bin_8:
        message_ascii.append(int(bits8, 2))
    # initialiser une nouvelle liste qui va contenir les caracteres du message encodes
    message_unicode = []
    for nbr in message_ascii:
        message_unicode.append(chr(nbr))
    # transformer la liste en une chaine de caractere
    message = ''
    for caractere in message_unicode:
        message += caractere
    return message

# programme principal
#traitement_image('/Users/roulaabifadel/Downloads/download (4).jpeg','ABCDEFGHIJKLMNOPQRSTUVWXYZ abcdef')
decodage('/Users/roulaabifadel/Downloads/download (4) (encodé).png')

'ABCDEFGHIJKLMNOPQRSTUVWXYZ abcdef'

In [15]:
L=[(9, 9), (0, 0), (1, 0), (2, 0), (8, 1), (9, 1), (0, 2), (7, 2), (8, 2), (9, 2), (7, 5), (8, 5), (9, 5), (9, 6), (0, 7), (1, 7), (3, 0), (4, 0), (5, 0), (9, 3), (0, 4), (1, 4), (0, 6), (1, 6), (2, 6), (6, 0), (7, 0), (8, 0), (2, 1), (3, 1), (4, 1), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (3, 6), (4, 6), (5, 6), (0, 9), (1, 9), (2, 9), (9, 0), (0, 1), (1, 1), (8, 4), (9, 4), (0, 5), (1, 5), (2, 5), (3, 5), (6, 6), (7, 6), (8, 6), (5, 1), (6, 1), (7, 1), (4, 5), (5, 5), (6, 5), (5, 7), (6, 7), (7, 7), (1, 8), (2, 8), (3, 8), (1, 2), (2, 2), (3, 2), (3, 3), (4, 3), (5, 3), (2, 7), (3, 7), (4, 7), (4, 2), (5, 2), (6, 2), (8, 7), (9, 7), (0, 8), (0, 3), (1, 3), (2, 3), (4, 8), (5, 8), (6, 8), (6, 3), (7, 3), (8, 3), (7, 8), (8, 8), (9, 8), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9)]



msg=''
image='/Users/roulaabifadel/Downloads/blanc10x10 (encodé).png'
im = Image.open(image)
im = im.convert('RGBA')
L1=[]
w_img = (im.size)[0]
h_img = (im.size)[1]
while L[0]!=(0,0):
    del L[0]
for loop in range(0,len(L),3):
    L1.append(L[loop])
    L1.append(L[loop+1])
for i in range(0,len(L1),2):
    binaire=''
    pix=im.getpixel(L1[i])
    pix2=im.getpixel(L1[i+1])
    for k in pix:
        binaire+=str(k%2)
    for k in pix2:
        binaire+=str(k%2)
    msg+=chr(int(binaire,2))
print(msg)

ABCDEFGHIJKLMNOPQRSTUVWXYZ abcdef


In [92]:
def bin_a_dec(binaire):
    Lbinaire=list(binaire)
    Lbinaire.reverse()
    decimal=0
    for i in range(len(Lbinaire)):
        decimal+=int(Lbinaire[i])*2**i
    return decimal
bin_a_dec('1111')


15